**Data Exploration**

In [ ]:
# Prcentage of data missing in combined dataset
all_data_na = (combined_data.isnull().sum() / len(combined_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

In [ ]:
# Skewness of the combined_data set
combined_data.agg(['skew']).transpose()

In [ ]:
# trying to deacrease the skew
np.log1p(combined_data["X.sales"]).agg(['skew']).transpose()

In [ ]:
train_data.info()

In [ ]:
train_data.nunique()

In [ ]:
for x in ["symbol", "description", "fee1", "fee2", "version"]:
  print(train_data[x].unique())

In [ ]:
from scipy.stats import skew, mode
# Missing value filling for fee1
agg_func_stats = {'fee1': [pd.Series.count, mode, pd.Series.mode]}
train_data.groupby(['cdate']).agg(agg_func_stats).reset_index()[0:40]

In [ ]:
# Missing value filling for fee2
agg_func_stats = {'version': [pd.Series.count, mode, pd.Series.mode]}
combined_data[combined_data["symbol"].isnull() == False]

In [ ]:
def lakshay_imputer(data,col_change):
    # new cases imputation using mode
    temp_impute = data[data[col_change].isnull()].index
    agg_func_stats = {col_change: [pd.Series.count, pd.Series.mode]}
    tmp = data.groupby(['cdate']).agg(agg_func_stats).reset_index()
    
    # for i in temp_impute: 
    #       print(data.iloc[i][col_change])
  
    for i in temp_impute: 
        if (tmp[tmp['cdate'] == data.iloc[i]['cdate']][(col_change,'count')].iloc[0] != 0.0):
          if (isinstance(tmp[tmp['cdate'] == data.iloc[i]['cdate']][[(col_change,'mode')]].iloc[0].iloc[0], float)):
            data.at[i, col_change ] = tmp[tmp['cdate'] == data.iloc[i]['cdate']][[(col_change,'mode')]].iloc[0].iloc[0]
          else:
            data.at[i, col_change ] = tmp[tmp['cdate'] == data.iloc[i]['cdate']][[(col_change,'mode')]].iloc[0].iloc[0][0]
        
    # for i in temp_impute: 
    #     print(data.iloc[i][col_change])

    return data

In [ ]:
data = combined_data.copy()
col_change = 'fee2'
data = lakshay_imputer(data,col_change)


col_change = 'fee1'
data = lakshay_imputer(data,col_change)
# Prcentage of data missing after imputation
all_data_na = (data.isnull().sum() / len(data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

In [ ]:
data1 = combined_data.copy()
# stop words
# removing words with count = 1 and the top N words
stry = list()
for mystring in data1['description'].fillna(""):
  result = mystring.split()
  stry = stry + result

from collections import Counter
final = Counter(stry)
df = pd.DataFrame.from_dict(final, orient='index').reset_index().rename(columns={'index':'word', 0:'count'})

# Get names of indexes for which column count has value 1
indexNames = df[ df['count'] == 1 ].index
stopwords = df.iloc[indexNames]
print(len(stopwords['word'].to_list()))
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

df.sort_values(by='count', ascending=False).reset_index().drop(['index'], axis = 1)
stopwords = stopwords['word'].to_list()

In [ ]:
df = combined_data.copy().fillna("")
df = df[['symbol','description']]
# concatenate the string
df['description'] = df.groupby(['symbol'])['description'].transform(lambda x : ' '.join(x))
  
# drop duplicate data
df = df.drop_duplicates()   
df = df.reset_index().drop(['index'], axis =1)
# # show the dataframe
# print(df[['symbol','description']])
# Idea for future:
# combine both datasets to make a more refine voculary
from sklearn.feature_extraction.text import CountVectorizer

#get the text column 
docs=df['description'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,ngram_range=(1, 1),lowercase=False,stop_words=frozenset(stopwords))
word_count_vector=cv.fit_transform(docs)

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=False)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
# you only needs to do this once, this is a mapping of index to 
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=combined_data['description'].fillna("").tolist()

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform(doc))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,100)

# now print the results
print("\n=====Doc=====")
#print(doc)
print("\n===Keywords===")
print(len(sorted_items))
somelist = [x for x in sorted_items if x[1] < 0.01]
for x in sorted_items:
    if(x[1] < 0.01):
        sorted_items.remove(x)
  
print(len(sorted_items)) 
print(len(somelist))
# for k in keywords:
#     print(k,keywords[k])
# stopwords = stopwords['word']
feature_vals = list()
for idx, score in somelist:
        feature_vals.append(feature_names[idx])
print(len(list(set((stopwords + feature_vals)))))
stopwords = stopwords + feature_vals
'''
Uses of this:
Now we can iterate it over every symbol we have to create a refrence and then compare 
every description that lacks a symbol. Then on the basis of similarity with our known numbers we say if a descriptio belongs to a symbol.
'''

In [ ]:
df = combined_data.copy()
df['description'] = df['description'].fillna("")
df = df[['symbol','description']]
# concatenate the string
# df['description'] = df.groupby(['symbol'])['description'].transform(lambda x : ' '.join(x))
  
# # drop duplicate data
# df = df.drop_duplicates()   
# df = df.reset_index().drop(['index'], axis =1)
# # show the dataframe
# print(df[['symbol','description']])
# Idea for future:
# combine both datasets to make a more refine voculary
from sklearn.feature_extraction.text import CountVectorizer

#get the text column 
docs=df['description'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,ngram_range=(1, 1),lowercase=False,stop_words=frozenset(stopwords))
word_count_vector=cv.fit_transform(docs)

print(len(list(cv.vocabulary_.keys())))
svd = TruncatedSVD(1000)
svd.fit(word_count_vector)
print(svd.explained_variance_ratio_.sum())

lsa = make_pipeline(svd, Normalizer(copy=False))
lsa_data = lsa.fit_transform(word_count_vector)
print(lsa_data.shape)
lsa_data = pd.DataFrame(lsa_data)
print(lsa_data.head())

#create dataframe
cv_dataframe=pd.DataFrame(lsa_data)
cv_dataframe['symbol'] = df['symbol']
print(cv_dataframe)

In [ ]:
# Store our results in a DataFrame
metrics = ['Algorithm','n_train','Features','ARI','Homogeneity',
           'Silhouette','Mutual_Info','Cross_Val','Train_Accuracy',
           'Test_Accuracy']
performance = pd.DataFrame(columns=metrics)
final = cv_dataframe
#final = pd.merge(combined_data[combined_data["symbol"].isnull() == False].fillna("")[['symbol']], cv_dataframe, on='symbol')

In [ ]:
# Add image data too
X = cv_dataframe[cv_dataframe["symbol"].isnull() == False] 
X[['fi1', 'fi2', 'fi3', 'fi4', 'fi5', 'fi6', 'fi7', 'FT0000', 'FT0001', 'FT0002', 'FT0003', 'FT0004', 'FT0005', 'FT0006',
       'FT0007', 'FT0008', 'FT0009']] = combined_data[['fi1', 'fi2', 'fi3', 'fi4', 'fi5', 'fi6', 'fi7', 'FT0000', 'FT0001', 'FT0002', 'FT0003', 'FT0004', 'FT0005', 'FT0006',
       'FT0007', 'FT0008', 'FT0009']]
y = X['symbol']
X = X.drop(['symbol'],axis = 1)

X_train, X_test, y_train, y_test = train_test_split (X,y,test_size=0.3)